# Load imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_STATE = 42

# Load Processed PCA datasets

### Clustering on PCA features
Using the PCA outputs from cluster_feature_selection.ipynb (Ryan). We search a few clustering families (KMeans, Agglomerative, Spectral, DBSCAN), rank by silhouette/CH/DB, and export cluster assignments + profiles for downstream analysis.

In [2]:

# Loading .CSV files created in cluster_feature_selection.ipynb (Ryan)
bat_path = "csv_files/batters_pca_features.csv"
pit_path = "csv_files/pitchers_pca_features.csv"

batters_df = pd.read_csv(bat_path)
pitchers_df = pd.read_csv(pit_path)

# identifier and non-feature columns (we keep Salary because it's already normalized per-year in Ryan's step)
id_col = "Name"
non_feature_cols = {id_col, "Team", "Year"}

# feature matrices for clustering
bat_feature_cols = [c for c in batters_df.columns if c not in non_feature_cols]
pit_feature_cols = [c for c in pitchers_df.columns if c not in non_feature_cols]

X_bat = batters_df[bat_feature_cols]
X_pit = pitchers_df[pit_feature_cols]

In [3]:
X_bat.head()

,Overall_Offensive_Production,Speed_and_Baserunning_Aggression,Contact_and_OnBase_Efficiency,Experience_and_Longevity,Plate_Toughness_and_Intentionality,Situational_Contact_Hitting,Patience_vs_Strikeout_Profile,Gap_Power_and_Doubles_Ability,Productive_Baserunning,Salary
0,6.000405,1.585444,0.357066,-1.641773,1.802005,-0.619777,1.604060,-1.885356,1.348817,1.925225
1,3.827357,1.994843,-1.001717,-0.846473,-0.548212,-1.556538,-0.326493,-0.565544,0.167803,2.168257
2,2.029447,2.107021,-0.420237,0.318406,-0.235169,-1.450602,0.242976,-0.147753,-1.532303,-1.792468
3,2.874128,-1.626197,0.261417,0.044747,-0.781510,0.662864,1.679883,-0.768395,0.616730,1.401482
4,2.729666,0.716261,1.491701,-0.626843,-2.687419,-1.362838,1.896120,0.079350,2.671523,-1.783172


In [4]:
X_pit.head()

,Workload_and_Volume,Run_Prevention,Command_and_Control,Stamina_and_CompleteGames,Bullpen_Leverage,Strikeout_Dominance,Veteran_Pitching_Style,Mechanical_Volatility,Pressure_Composure,Relief_Effectiveness,Overall_Performance_Value,Sabermetric_Efficiency,Salary
0,3.262113,0.416926,-0.530285,-1.738596,-0.582734,0.328378,0.454630,-0.876445,0.215656,-1.040292,0.827779,0.432365,5.621204
1,3.262113,0.416926,-0.530285,-1.738596,-0.582734,0.328378,0.454630,-0.876445,0.215656,-1.040292,0.827779,0.432365,5.621204
2,2.857388,3.757410,1.192545,-0.307011,-1.208139,0.629566,0.394610,0.734392,1.362479,0.010549,-0.845647,0.232409,1.887012
3,2.857388,3.757410,1.192545,-0.307011,-1.208139,0.629566,0.394610,0.734392,1.362479,0.010549,-0.845647,0.232409,1.887012
4,2.749648,0.480162,-1.790568,1.783913,0.037652,-0.115295,1.000747,0.089898,0.405998,0.330040,-0.251725,-0.152107,3.838537


# Define Clustering Utilities

We evaluate several clustering approaches. KMeans is our primary baseline (Ryan’s PCA space is well-suited). Spectral acts as a kernelized KMeans analogue. Agglomerative provides a hierarchical check. DBSCAN gives a density-based view. We report silhouette, Calinski–Harabasz, Davies–Bouldin.

In [5]:
def score_clusters(X, labels):
    """Return common clustering metrics; handle single/noise cluster edge cases."""
    n = len(set(labels)) - (1 if -1 in labels else 0)
    if n < 2:
        return {"silhouette": np.nan, "calinski": np.nan, "davies": np.nan, "n_clusters": n}
    return {
        "silhouette": silhouette_score(X, labels),
        "calinski": calinski_harabasz_score(X, labels),
        "davies": davies_bouldin_score(X, labels),
        "n_clusters": n,
    }

def run_kmeans_grid(X, ks=range(3, 11), random_state=RANDOM_STATE):
    rows = []
    for k in ks:
        model = KMeans(n_clusters=k, n_init="auto", random_state=random_state)
        labels = model.fit_predict(X)
        rows.append(score_clusters(X, labels) | {"method": "kmeans", "k": k, "model": model})
    return pd.DataFrame(rows)

def run_agglomerative_grid(X, ks=range(3, 11)):
    rows = []
    for k in ks:
        model = AgglomerativeClustering(n_clusters=k)
        labels = model.fit_predict(X)
        rows.append(score_clusters(X, labels) | {"method": "agglomerative", "k": k, "model": model})
    return pd.DataFrame(rows)

def run_spectral_grid(X, ks=range(3, 11), random_state=RANDOM_STATE):
    rows = []
    for k in ks:
        model = SpectralClustering(n_clusters=k, assign_labels="kmeans", affinity="rbf", random_state=random_state)
        labels = model.fit_predict(X)
        rows.append(score_clusters(X, labels) | {"method": "spectral", "k": k, "model": model})
    return pd.DataFrame(rows)

def run_dbscan_grid(X, eps_list=(0.5, 1.0, 1.5, 2.0), min_samples_list=(5, 10)):
    rows = []
    for eps in eps_list:
        for ms in min_samples_list:
            model = DBSCAN(eps=eps, min_samples=ms)
            labels = model.fit_predict(X)
            rows.append(score_clusters(X, labels) | {"method": "dbscan", "eps": eps, "min_samples": ms, "model": model})
    return pd.DataFrame(rows)

def rank_models(search_df):
    """Rank by silhouette (desc), then Calinski–Harabasz (desc), then Davies–Bouldin (asc)."""
    d = search_df.copy()
    d["davies_inv"] = -d["davies"].fillna(np.inf)
    d = (
        d.assign(
            r_sil=lambda t: t["silhouette"].rank(ascending=False, method="min"),
            r_cal=lambda t: t["calinski"].rank(ascending=False, method="min"),
            r_dav=lambda t: t["davies_inv"].rank(ascending=False, method="min"),
        )
        .assign(rank_score=lambda t: t.r_sil * 100 + t.r_cal * 0.01 + t.r_dav * 0.0001)
        .sort_values("rank_score")
    )
    return d

# Grid Search Across Methods

Search K=3…10 for the centroid/hierarchical models; small grids for DBSCAN (eps ∈ {0.5,1.0,1.5,2.0}, min_samples ∈ {5,10}). Save complete metrics for both datasets.

In [6]:
bat_search = pd.concat(
    [
        run_kmeans_grid(X_bat),
        run_agglomerative_grid(X_bat),
        run_spectral_grid(X_bat),
        run_dbscan_grid(X_bat),
    ],
    ignore_index=True,
)

pit_search = pd.concat(
    [
        run_kmeans_grid(X_pit),
        run_agglomerative_grid(X_pit),
        run_spectral_grid(X_pit),
        run_dbscan_grid(X_pit),
    ],
    ignore_index=True,
)

# Persist metrics for review
bat_search.to_csv("clustering_metrics_batters.csv", index=False)
pit_search.to_csv("clustering_metrics_pitchers.csv", index=False)

# quick look at top configs by silhouette
display(bat_search.sort_values("silhouette", ascending=False).head(10))
display(pit_search.sort_values("silhouette", ascending=False).head(10))

,silhouette,calinski,davies,n_clusters,method,k,model,eps,min_samples
17,0.311729,11.167753,0.581635,4,spectral,4.0,"SpectralClustering(n_clusters=4, random_state=42)",NaN,NaN
16,0.311430,10.781713,0.592830,3,spectral,3.0,"SpectralClustering(n_clusters=3, random_state=42)",NaN,NaN
18,0.301381,10.608698,0.577023,5,spectral,5.0,"SpectralClustering(n_clusters=5, random_state=42)",NaN,NaN
19,0.288383,11.885918,0.545597,6,spectral,6.0,"SpectralClustering(n_clusters=6, random_state=42)",NaN,NaN
21,0.287258,13.742744,0.827643,8,spectral,8.0,SpectralClustering(random_state=42),NaN,NaN
23,0.286551,11.344629,0.627058,10,spectral,10.0,"SpectralClustering(n_clusters=10, random_state...",NaN,NaN
20,0.272534,13.287926,0.572523,7,spectral,7.0,"SpectralClustering(n_clusters=7, random_state=42)",NaN,NaN
22,0.254111,13.388384,0.811604,9,spectral,9.0,"SpectralClustering(n_clusters=9, random_state=42)",NaN,NaN
8,0.221365,1572.977978,1.792340,3,agglomerative,3.0,AgglomerativeClustering(n_clusters=3),NaN,NaN
1,0.214958,1662.714251,1.479975,4,kmeans,4.0,"KMeans(n_clusters=4, random_state=42)",NaN,NaN


,silhouette,calinski,davies,n_clusters,method,k,model,eps,min_samples
19,0.324024,21.569186,1.246476,6,spectral,6.0,"SpectralClustering(n_clusters=6, random_state=42)",NaN,NaN
16,0.278244,20.690584,0.487519,3,spectral,3.0,"SpectralClustering(n_clusters=3, random_state=42)",NaN,NaN
18,0.260249,60.663796,0.670014,5,spectral,5.0,"SpectralClustering(n_clusters=5, random_state=42)",NaN,NaN
17,0.251878,18.860124,0.577397,4,spectral,4.0,"SpectralClustering(n_clusters=4, random_state=42)",NaN,NaN
22,0.212038,18.010895,1.161627,9,spectral,9.0,"SpectralClustering(n_clusters=9, random_state=42)",NaN,NaN
21,0.211789,18.440922,1.137095,8,spectral,8.0,SpectralClustering(random_state=42),NaN,NaN
20,0.211338,19.123345,1.227693,7,spectral,7.0,"SpectralClustering(n_clusters=7, random_state=42)",NaN,NaN
23,0.209799,16.354484,0.814636,10,spectral,10.0,"SpectralClustering(n_clusters=10, random_state...",NaN,NaN
8,0.204348,1263.753005,1.754005,3,agglomerative,3.0,AgglomerativeClustering(n_clusters=3),NaN,NaN
9,0.164885,1073.037531,1.836483,4,agglomerative,4.0,AgglomerativeClustering(n_clusters=4),NaN,NaN


# Fit Best Models and Export Results

### Model selection and export

Pick the top-ranked config per dataset, fit once more, then persist two artifacts:

cluster assignments (Name → cluster)

cluster profiles (feature means by cluster)

In [7]:
def fit_best_and_export(search_df, X, raw_df, label, out_prefix):
    ranked = rank_models(search_df)
    best = ranked.iloc[0]
    model = best["model"]
    clusters = model.fit_predict(X)

    # assignments (Name -> cluster)
    assign = raw_df[[id_col]].copy()
    assign["cluster"] = clusters
    assign_path = f"{out_prefix}_{label}.csv"
    assign.to_csv(assign_path, index=False)

    # cluster profiles (mean of features by cluster)
    prof = pd.DataFrame(X, columns=X.columns if isinstance(X, pd.DataFrame) else None).copy()
    prof["cluster"] = clusters
    prof = prof.groupby("cluster").mean().reset_index()
    prof_path = f"cluster_profiles_{label}.csv"
    prof.to_csv(prof_path, index=False)

    print(f"[{label}] best:", {k: best.get(k) for k in ["method", "k", "eps", "min_samples", "n_clusters"]})
    print("saved:", assign_path, "and", prof_path)
    return assign, prof, best

bat_assign, bat_prof, bat_best = fit_best_and_export(
    bat_search, X_bat, batters_df, label="batters", out_prefix="batters_clusters"
)
pit_assign, pit_prof, pit_best = fit_best_and_export(
    pit_search, X_pit, pitchers_df, label="pitchers", out_prefix="pitchers_clusters"
)

display(bat_assign.head())
display(pit_assign.head())

[batters] best: {'method': 'spectral', 'k': np.float64(4.0), 'eps': np.float64(nan), 'min_samples': np.float64(nan), 'n_clusters': np.int64(4)}
saved: batters_clusters_batters.csv and cluster_profiles_batters.csv
[pitchers] best: {'method': 'spectral', 'k': np.float64(6.0), 'eps': np.float64(nan), 'min_samples': np.float64(nan), 'n_clusters': np.int64(6)}
saved: pitchers_clusters_pitchers.csv and cluster_profiles_pitchers.csv


,Name,cluster
0,justin upton,0
1,chris young,0
2,ryan roberts,0
3,miguel montero,0
4,gerardo parra,0


,Name,cluster
0,nathan eovaldi,0
1,nathan eovaldi,0
2,nick pivetta,0
3,nick pivetta,0
4,michael wacha,0


# Visualize Clusters

### Quick visualization

2D scatter for a fast sanity check. Axes pick meaningful PCA features when available, else the first two.

In [ ]:
def quick_scatter(df, x, y, hue_col="cluster", title=""):
    plt.figure(figsize=(6, 5))
    sns.scatterplot(data=df, x=x, y=y, hue=hue_col, s=30)
    plt.title(title)
    plt.tight_layout()
    plt.show()

# choose sensible axes by name; fall back to the first two features
bat_x = next((c for c in X_bat.columns if "Offensive" in c), X_bat.columns[0])
bat_y = next((c for c in X_bat.columns if "Speed" in c), X_bat.columns[1])

pit_x = next((c for c in X_pit.columns if "Workload" in c), X_pit.columns[0])
pit_y = next((c for c in X_pit.columns if "Run_Prevention" in c), X_pit.columns[1])

bat_vis = batters_df[[id_col, "Team", "Year"]].merge(bat_assign, on=id_col, how="left").join(X_bat)
pit_vis = pitchers_df[[id_col, "Team", "Year"]].merge(pit_assign, on=id_col, how="left").join(X_pit)

quick_scatter(bat_vis, bat_x, bat_y, "cluster", "Batters — clusters (best model)")
quick_scatter(pit_vis, pit_x, pit_y, "cluster", "Pitchers — clusters (best model)")

# Team x Cluster Counts

For Charles if helpful: per team counts by cluster for feeding a downstream regression

In [ ]:
if "Team" in batters_df.columns:
    bat_team_counts = (
        bat_vis.pivot_table(index="Team", columns="cluster", aggfunc="size", fill_value=0)
        .sort_index(axis=1)
    )
    bat_team_counts.to_csv("team_cluster_counts_batters.csv")

if "Team" in pitchers_df.columns:
    pit_team_counts = (
        pit_vis.pivot_table(index="Team", columns="cluster", aggfunc="size", fill_value=0)
        .sort_index(axis=1)
    )
    pit_team_counts.to_csv("team_cluster_counts_pitchers.csv")